In [987]:
# Importando librerias
import pandas as pd
import numpy as np
import copy 
import random


In [988]:
# Funcion para la importacion de los datos 

def importar_ejercicio(ejercicios):
    dataframe = pd.read_excel(r'.\problemset\problemset.xlsx')
    cost_gasolina = float(dataframe['fuel_price'][ejercicios])
    lista_comp =  dataframe['query'].iloc[ejercicios]
    
    # Limpiando la data
    
    lista_comp = lista_comp.replace('[', '')
    lista_comp = lista_comp.replace(']', '')
    lista_comp = lista_comp.replace("'",'')
    
    # Convirtiendo a una lista
    
    lista_comp = lista_comp.split(',')
    for i in lista_comp[1:]:
        lista_comp[lista_comp.index(i)] = i[1:]
    
    # Importanddo las hojas de excel 
    
    catalogo = dataframe['catalog'].iloc[ejercicios] 
    catalogo = pd.read_excel(catalogo, sheet_name='catalogo')
    graph = dataframe['catalog'].iloc[ejercicios]
    graph = pd.read_excel(graph, sheet_name='grafo')
    
    return cost_gasolina, lista_comp, catalogo, graph

In [989]:
cost_gasolina, lista_comp, catalogo, graph = importar_ejercicio (18)

print(f"""
Costo de la gasolina:
{cost_gasolina}

Catalogo:
{catalogo.head()}

Lista de compra:
{lista_comp}

Grafo:
{graph}
""")


Costo de la gasolina:
9.0

Catalogo:
                   store         product    price
0                  Jumbo   Jamón de pavo  3982.99
1             Super Pola        Albahaca  4388.99
2  Supermercado Nacional     Televisores   728.99
3  Supermercado Nacional      Aftershave  4306.99
4              La Sirena  Ropa de hombre  1611.99

Lista de compra:
['Aftershave', 'Chicles de fresa', 'Ropa de hombre', 'Televisores', 'Refresco de cola', 'Fósforos', 'Jamón de pavo']

Grafo:
                   edge 1                 edge 2  distance
0               La Sirena                   Casa         0
1                   Bravo  Supermercado Nacional        10
2   Supermercado Nacional  Supermercado Nacional         9
3              Super Pola  Supermercado Nacional         1
4              Super Pola                  Jumbo         2
5               La Sirena             Super Pola         2
6                    Casa                  Jumbo         9
7                   Bravo                   Cas

In [990]:
def crear_matriz_distancia(graph):
    tiendas_unicas = graph['edge 1'].unique().tolist()
    matrix = pd.DataFrame(index=tiendas_unicas, columns=tiendas_unicas)
    for _, row in graph.iterrows():
        matrix.loc[row['edge 1'], row['edge 2']] = row['distance']

    for edge1 in matrix.index:
        for edge2 in matrix.columns:
            if pd.isna(matrix.loc[edge1, edge2]):
                if not pd.isna(matrix.loc[edge2, edge1]):
                    matrix.loc[edge1, edge2] = matrix.loc[edge2, edge1]
    ''' 
        Maestros, Me di cuenta que la diagonal principal estaba mal, Y agregue esta linea, para que la diagonal principal sea 0, Y se forme una grafo
        totalmente conexo. 

        Ademas a eso, habian valores faltantes, asi que los rellene tomando en cuenta los valores de rutas similares
        ejemplo de ruta similar, La Sirena a Casa hay una distancia de 4 pero de Casa a La Sirena no hay distancia, asi que se 
        toma la distancia de La Sirena a Casa.
    '''
    np.fill_diagonal(matrix.values, 0) 

    return matrix


In [991]:
distance_matrix = crear_matriz_distancia(graph)
distance_matrix

,La Sirena,Bravo,Supermercado Nacional,Super Pola,Casa,Jumbo
La Sirena,0,8,5,2,0,8
Bravo,8,0,10,1,5,17
Supermercado Nacional,6,15,0,18,8,11
Super Pola,1,16,1,0,5,2
Casa,0,3,8,17,0,9
Jumbo,8,17,18,13,1,0


In [992]:
catalogo.head()

,store,product,price
0,Jumbo,Jamón de pavo,3982.99
1,Super Pola,Albahaca,4388.99
2,Supermercado Nacional,Televisores,728.99
3,Supermercado Nacional,Aftershave,4306.99
4,La Sirena,Ropa de hombre,1611.99


In [993]:
lista_comp

['Aftershave',
 'Chicles de fresa',
 'Ropa de hombre',
 'Televisores',
 'Refresco de cola',
 'Fósforos',
 'Jamón de pavo']

In [994]:
# Poblador inicial
def creando_pobladores(n_pobladores):
    pobladores = []
    for n in range(n_pobladores):
        poblador = {'Casa':'[]'}
        productos = catalogo['product']
        aux = []
        for elemento in lista_comp:
            productos_filtrados = productos[productos == elemento]
            idx = np.random.choice(productos_filtrados.index)
            aux.append(catalogo['store'][idx] + "," + catalogo['product'][idx] + "," + str(catalogo['price'][idx]))

        for elemento in aux:
            list_elementos = elemento.split(',')
            poblador[list_elementos[0]] = []

        poblador['Precio'] = 0

        for elemento in aux:
            list_elementos = elemento.split(',')
            poblador[list_elementos[0]].append(list_elementos[1])
            poblador['Precio'] += float(list_elementos[2])
        pobladores.append(poblador)

    for poblador in pobladores:
        cost_viaje = 0
        keys = list(poblador.keys())[:-1]
        for tienda in range(len(keys)):
            if keys[tienda] != keys[-1]:
                cost_viaje += distance_matrix.at[keys[tienda], keys[tienda+1]]
                cost_viaje *= cost_gasolina 
                poblador["Precio"] += cost_viaje

        poblador["Precio"] += (distance_matrix.at[keys[-1], 'Casa'] * cost_gasolina)
    
    return pobladores

In [995]:
pobladores = creando_pobladores(20)

In [996]:
pobladores

[{'Casa': '[]',
  'Supermercado Nacional': ['Aftershave', 'Chicles de fresa', 'Televisores'],
  'Bravo': ['Ropa de hombre', 'Fósforos', 'Jamón de pavo'],
  'Jumbo': ['Refresco de cola'],
  'Precio': 21678.93},
 {'Casa': '[]',
  'Bravo': ['Aftershave'],
  'Jumbo': ['Chicles de fresa', 'Ropa de hombre'],
  'La Sirena': ['Televisores', 'Refresco de cola', 'Fósforos'],
  'Supermercado Nacional': ['Jamón de pavo'],
  'Precio': 53170.93},
 {'Casa': '[]',
  'Bravo': ['Aftershave', 'Refresco de cola'],
  'Jumbo': ['Chicles de fresa'],
  'La Sirena': ['Ropa de hombre', 'Fósforos', 'Jamón de pavo'],
  'Supermercado Nacional': ['Televisores'],
  'Precio': 48223.93},
 {'Casa': '[]',
  'Super Pola': ['Aftershave', 'Ropa de hombre'],
  'Supermercado Nacional': ['Chicles de fresa'],
  'La Sirena': ['Televisores', 'Jamón de pavo'],
  'Bravo': ['Refresco de cola', 'Fósforos'],
  'Precio': 137912.93},
 {'Casa': '[]',
  'La Sirena': ['Aftershave', 'Jamón de pavo'],
  'Super Pola': ['Chicles de fresa'],
 

In [1019]:
def fitness(pobladores):
    pobladores_ordenados = sorted(pobladores, key=lambda x: x['Precio'])
    minimos_diferentes = []
    for item in pobladores_ordenados:
        if len(minimos_diferentes) < 2 and (not minimos_diferentes or item['Precio'] != minimos_diferentes[-1]['Precio']):
            minimos_diferentes.append(item)
    
    return minimos_diferentes

padre,madre = fitness(pobladores)

In [998]:
padre

{'Casa': '[]',
 'La Sirena': ['Aftershave', 'Jamón de pavo'],
 'Super Pola': ['Chicles de fresa'],
 'Supermercado Nacional': ['Ropa de hombre',
  'Televisores',
  'Refresco de cola',
  'Fósforos'],
 'Precio': 20265.93}

In [999]:
madre

{'Casa': '[]',
 'Supermercado Nacional': ['Aftershave', 'Chicles de fresa', 'Televisores'],
 'Bravo': ['Ropa de hombre', 'Fósforos', 'Jamón de pavo'],
 'Jumbo': ['Refresco de cola'],
 'Precio': 21678.93}

In [1002]:
def crossover(padre, madre, N):

    keys1 = list(padre.keys())
    keys2 = list(madre.keys())

    # Eliminar la clave 'Precio' de las listas de claves
    keys1.remove('Precio')
    keys2.remove('Precio')

    i = 0
    while i < len(keys1) and i < len(keys2):
        key1 = keys1[i]
        key2 = keys2[i]

        # Si cualquiera de los diccionarios se queda sin productos, detener el intercambio
        if not padre[key1] or not madre[key2]:
            break
        
        # Intercambiar los productos de las tiendas actuales
        padre[key1], madre[key2] = madre[key2], padre[key1]
        
        # Eliminar la clave de la lista si el diccionario ya no tiene más productos en esa tienda
        if not padre[key1]:
            keys1.remove(key1)
        if not madre[key2]:
            keys2.remove(key2)
        
        i += 1

    hijo1 = padre
    hijo2 = madre
    hijos = []
    
    for i in range(N):
        # Alternar entre hijo1 y hijo2
        if i % 2 == 0:
            base = copy.deepcopy(hijo1)
        else:
            base = copy.deepcopy(hijo2)
        
        # Obtener las claves de las tiendas, excluyendo 'Casa' y 'Precio'
        tiendas = [clave for clave in base.keys() if clave not in ['Casa', 'Precio']]
        
        # Si hay al menos dos tiendas, intercambiar productos entre dos tiendas aleatorias
        if len(tiendas) > 1:
            tienda1, tienda2 = random.sample(tiendas, 2)
            base[tienda1], base[tienda2] = base[tienda2], base[tienda1]
        
        # Agregar el nuevo hijo a la lista de hijos
        hijos.append(base)
    
    return hijos

In [1003]:
hijos = crossover(padre,madre,20)

In [1004]:
def actualizar_precios(lista_diccionarios, catalogo):
    for diccionario in lista_diccionarios:
        precio_total = 0
        
        for tienda, productos in diccionario.items():
            if tienda == 'Casa' or tienda == 'Precio':
                continue
            
            # Verificar si la tienda se encuentra en el catalogo
            if tienda in catalogo['store'].values:
                for producto in productos:
                    # Buscar el precio del producto en la tienda correspondiente
                    precio_producto = catalogo.loc[(catalogo['store'] == tienda) & 
                                                   (catalogo['product'] == producto), 'price']
                    if not precio_producto.empty:
                        precio_total += precio_producto.values[0]
        
        # Actualizar el precio total en el diccionario
        diccionario['Precio'] = precio_total
    
    for poblador in lista_diccionarios:
        cost_viaje = 0
        keys = list(poblador.keys())[:-1]
        for tienda in range(len(keys)):
            if keys[tienda] != keys[-1]:
                cost_viaje += distance_matrix.at[keys[tienda], keys[tienda+1]]
                cost_viaje *= cost_gasolina 
                poblador["Precio"] += cost_viaje

        poblador["Precio"] += (distance_matrix.at[keys[-1], 'Casa'] * cost_gasolina)
    
    return lista_diccionarios

In [1012]:
nueva_generacion = actualizar_precios(hijos , catalogo)

In [1013]:
len(nueva_generacion)

20

In [1014]:
def validacion(lista_de_diccionarios):
    filtrada_lista_de_diccionarios = []

    for dic in lista_de_diccionarios:
        valido = True
        for tienda, productos in dic.items():
            if tienda == 'Precio' or tienda == 'Casa':
                continue
            for producto in productos:
                if not ((catalogo['product'] == producto) & (catalogo['store'] == tienda)).any():
                    valido = False
                    break
            if not valido:
                break
        if valido:
            filtrada_lista_de_diccionarios.append(dic)

    return filtrada_lista_de_diccionarios

In [1016]:
len(validacion(nueva_generacion))

5

In [1017]:
nueva_generacion = validacion(nueva_generacion)

In [1020]:
fitness(nueva_generacion)

[{'Casa': '[]',
  'La Sirena': ['Ropa de hombre',
   'Televisores',
   'Refresco de cola',
   'Fósforos'],
  'Super Pola': ['Chicles de fresa'],
  'Supermercado Nacional': ['Aftershave', 'Jamón de pavo'],
  'Precio': 17692.93},
 {'Casa': '[]',
  'Supermercado Nacional': ['Ropa de hombre', 'Fósforos', 'Jamón de pavo'],
  'Bravo': ['Aftershave', 'Chicles de fresa', 'Televisores'],
  'Jumbo': ['Refresco de cola'],
  'Precio': 31316.929999999997}]

In [1021]:
cost_gasolina

9.0

In [1023]:
distance_matrix

,La Sirena,Bravo,Supermercado Nacional,Super Pola,Casa,Jumbo
La Sirena,0,8,5,2,0,8
Bravo,8,0,10,1,5,17
Supermercado Nacional,6,15,0,18,8,11
Super Pola,1,16,1,0,5,2
Casa,0,3,8,17,0,9
Jumbo,8,17,18,13,1,0


In [1022]:
catalogo

,store,product,price
0,Jumbo,Jamón de pavo,3982.99
1,Super Pola,Albahaca,4388.99
2,Supermercado Nacional,Televisores,728.99
3,Supermercado Nacional,Aftershave,4306.99
4,La Sirena,Ropa de hombre,1611.99
5,La Sirena,Aftershave,1702.99
6,La Sirena,Jamón de pavo,3339.99
7,Jumbo,Refresco de cola,1015.99
8,Supermercado Nacional,Rizadores de pelo,3197.99
9,Supermercado Nacional,Jamón de pavo,4785.99


1- Se importa la data necesaria para resolver el problema.
  * se crea una matriz de distancia
      * Se rellenan valores faltantes con valores de rutas similares
      * Se rellena la diagonal principal con 0 

2- se crean poblbladres iniciales, la cantidad de pobladores viene dada por el hiperparametro n_pobladres
  * esos pobladres iniciales, se crean de forma aleatoria, tomando en cuenta la lista de compra y el catalogo
  * Innmediatamente se creen los pobladores, se le aniade el precio de estos sin tomar en cuenta la gasolina.
  * Tomando en cuenta las distancias de la ruta de cada poblador, se saca el coste de la gasolia a gastar y se le suma a el precio del poblador

3- Se aplica la funcion fitness
  * Ordena la lista de pobladores de menor a mayor 
  * Toma los dos primeros que sean diferentes 
  * Retorna estos dos como los mejores pobladores de la generacion 

4- Se cruzan los dos mejores pobladores de la generacion, utilizando el siguiente metodo de cruce para evitar perdida de datos y rutas erroneas:
  * Asi se ven los padres (Los mejores pobladores de la generacion)
    padre
    {'Casa': '[]',
      'La Sirena': ['Ropa de hombre',
      'Televisores',
      'Refresco de cola',
      'Fósforos'],
      'Super Pola': ['Chicles de fresa'],
      'Supermercado Nacional': ['Aftershave', 'Jamón de pavo'],
      'Precio': 17692.93}

    madre
    {'Casa': '[]',
      'Supermercado Nacional': ['Ropa de hombre', 'Fósforos', 'Jamón de pavo'],
      'Bravo': ['Aftershave', 'Chicles de fresa', 'Televisores'],
      'Jumbo': ['Refresco de cola'],
      'Precio': 31316.929999999997}

  * Se cruzan estos dos intercambiando la ruta de compra de uno con la ruta de compra del otro:
    hijo 1 (Este toma la ruta del padre y el orden en que compra la madre)
    {'Casa': '[]',
      'La Sirena': ['Aftershave', 'Chicles de fresa', 'Televisores'],
      'Super Pola': ['Ropa de hombre', 'Fósforos', 'Jamón de pavo'],
      'Supermercado Nacional': ['Refresco de cola'],
      'Precio': 17692.93}

    hijo 2 (Este tomaria la ruta de compra de la madre con el orden de compra del padre)
    ...
  
  * para lograr a cantidad de pobladores que habia al inicio, se intercambian el orden de compra para cada hijo hasta crear la misma cantidad que pobladores iniciales.
    Ejemplo con Hijo 1: 
    {'Casa': '[]',
      'La Sirena': ['Ropa de hombre', 'Fósforos', 'Jamón de pavo'],
      'Super Pola': ['Aftershave', 'Chicles de fresa', 'Televisores'],
      'Supermercado Nacional': ['Refresco de cola'],
      'Precio': 17692.93}

5- Se validan las listas de compras:
  * Se verifica que para cada nuevo poblador hayan productos en la tienda que este compra, si no hay, se elimina
  * Se retornan los pobladores filtrados

6- Se actualizan los precios:
  * Luego de filtrar los pobladores, actualizamos los precios de estos, incluyendo el coste de gasolina 
  * Se retorna la lista de pobladores con sus precios correspondientes 

7- Remplazo:
  * Estos nuevos pobladores estan listos para someterse a la funcion fitness, extraer los dos mejores y repetir el ciclo cuantas veces sea necesario.
